In [ ]:
import os
os.environ["OPENAI_API_KEY"]="YOUR_API_KEY"

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

CHROMA_PATH = ".\\chroma5"

In [4]:
# embeddings = OpenAIEmbeddings()
loaders = PyPDFLoader("C:\\Users\\ADMIN\\Downloads\\Bhagavad-gita-Swami-BG-Narasingha.pdf")

documents = loaders.load()

db = Chroma.from_documents(
    documents,
    OpenAIEmbeddings(),
    persist_directory=CHROMA_PATH
  )
db.persist()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_39712\2022741640.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [5]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [10]:
def query_rag(query_text):
  """
  Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
  Args:
    - query_text (str): The text to query the RAG system with.
  Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
  """
  # YOU MUST - Use same embedding function as before
  embedding_function = OpenAIEmbeddings()

  # Prepare the database
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
  
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=3)

  # Check if there are any matching results or if the relevance score is too low
  # if len(results) == 0 or results[0][1] < 0.7:
  #   print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)
  
  # Initialize OpenAI chat model
  model = ChatOpenAI()

  # Generate response text based on the prompt
  response_text = model.predict(prompt)
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources}"
  return formatted_response, response_text


In [11]:
query_text="what krishna told to Arjuna in chapter 2"
formatted_response, response_text = query_rag(query_text)
print(response_text)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_39712\3764950876.py:34: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_text = model.predict(prompt)


Based on the above context, Krishna explained to Arjuna in Chapter 2 the greatest secret about His supreme nature, which helped to remove Arjuna's bewilderment.


In [12]:
query_text="When was kurukshetra started?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Kurukshetra started in the month of December, in 3138 BCE.
